# Exercise Set 7: Data structuring  3

*Morning, August 15, Morning*

In this Exercise Set we finalize our work with the weather data we started working on in Exercise Set 4. We will also study a dataset of traffic data from Copenhagen to iterate through the pandas workflow once more. 

In [189]:
%matplotlib inline 

import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib as mpl

# Increases the plot size a little
mpl.rcParams['figure.figsize'] = 11, 6


## Exercise Section 7.1: Weather data, part 3
We continue with the final part of three exercises on structuring weather data. In this exercise you must use the function for fetching and structuring weather data which you made in Exercise 6.1.5.

> **Ex. 7.1.1:** Plot the monthly max,min, mean, first and third quartiles for maximum temperature for our station with the ID _'ITE00100550'_ in 1864. 

> *Hint*: the method `describe` computes all these measures.

In [198]:
def weather(x,y):
    url='https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/{}.csv.gz'.format(x)
    df_weather = pd.read_csv(url,
                         compression='gzip',
                         header=None).iloc[:,:4]
    df_weather.columns = ['station', 'datetime', 'obs_type', 'obs_value']
    df_weather['obs_value'] = df_weather['obs_value'] / 10
    df_select = df_weather[(df_weather.obs_type == 'TMAX')].copy()
    df_select['TMAX_F'] = 32 + 1.8 * df_select['obs_value']
    df_sorted = df_select.sort_values(by=['obs_value']).reset_index(drop=True)
    
    df_sorted['datetime']=df_sorted['datetime'].astype(str)
    df_sorted_time=df_sorted.copy()
    df_sorted_time['datetime']=pd.to_datetime(df_sorted['datetime'])
    df_sorted_time['Month']=df_sorted_time['datetime'].dt.month
   
    df_done=df_sorted_time.set_index('datetime')
    
    df_cnt=df_done.copy()
    df_cnt['Country']=df_done['station'].str[:3]
    
    df_x=df_cnt.copy()
    if y!='':
        df_y=df_x[(df_x.station == y)]
        return df_y
    else:
        return df_x

ite=weather(1864,'ITE00100550')

ite_desc=ite.groupby('Month')['obs_value'].describe()[['mean', 'min', 'max','25%','75%']]
ite_desc.plot()

> **Ex. 7.1.2:** Get the processed data from years 1864-1867 as a list of DataFrames. Convert the list into a single DataFrame by concatenating vertically. 

In [210]:
com=pd.DataFrame()
plist=[]

# DataFrame append approach
for i in range(1864,1868):
    com=com.append(weather(i,''))
    
# Concat approach
for i in range(1864,1868):
    plist.append(weather(i,''))
    
ver=pd.concat(plist, join='outer', axis=0)
ver=ver.reset_index()

In [211]:
print(ver)

        datetime      station obs_type  obs_value  TMAX_F  Month Country
0     1864-02-26  SZ000006717     TMAX      -34.0  -29.20      2     SZ0
1     1864-02-08  SZ000006717     TMAX      -17.4    0.68      2     SZ0
2     1864-02-17  CA006158350     TMAX      -16.7    1.94      2     CA0
3     1864-01-03  SZ000006717     TMAX      -16.5    2.30      1     SZ0
4     1864-01-02  CA006158350     TMAX      -16.1    3.02      1     CA0
5     1864-02-10  SZ000006717     TMAX      -15.9    3.38      2     SZ0
6     1864-01-14  EZE00100082     TMAX      -15.1    4.82      1     EZE
7     1864-02-09  SZ000006717     TMAX      -15.0    5.00      2     SZ0
8     1864-02-19  SZ000006717     TMAX      -14.7    5.54      2     SZ0
9     1864-02-05  SZ000006717     TMAX      -14.4    6.08      2     SZ0
10    1864-02-20  SZ000006717     TMAX      -14.2    6.44      2     SZ0
11    1864-01-16  EZE00100082     TMAX      -14.2    6.44      1     EZE
12    1864-02-18  CA006158350     TMAX      -13.9  

> **Ex. 7.1.3:** Parse the station location data which you can find at https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt. Merge station locations onto the weather data spanning 1864-1867.  

> _Hint:_ The location data have the folllowing format, 

```
------------------------------
Variable   Columns   Type
------------------------------
ID            1-11   Character
LATITUDE     13-20   Real
LONGITUDE    22-30   Real
ELEVATION    32-37   Real
STATE        39-40   Character
NAME         42-71   Character
GSN FLAG     73-75   Character
HCN/CRN FLAG 77-79   Character
WMO ID       81-85   Character
------------------------------
```

> *Hint*: The station information has fixed width format - does there exist a pandas reader for that?

In [222]:
url='https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt'
loc=pd.read_fwf(url, header=None)

In [219]:
loc.head()

0        1        2     3                      4    5    6  \
0  ACW00011604  17.1167 -61.7833  10.1  ST JOHNS COOLIDGE FLD  NaN  NaN   
1  ACW00011647  17.1333 -61.7833  19.2               ST JOHNS  NaN  NaN   
2  AE000041196  25.3330  55.5170  34.0    SHARJAH INTER. AIRP  NaN  GSN   
3  AEM00041194  25.2550  55.3640  10.4             DUBAI INTL  NaN  NaN   
4  AEM00041217  24.4330  54.6510  26.8         ABU DHABI INTL  NaN  NaN   

         7  
0      NaN  
1      NaN  
2  41196.0  
3  41194.0  
4  41217.0

In [220]:
ver.head()

datetime      station obs_type  obs_value  TMAX_F  Month Country
0 1864-02-26  SZ000006717     TMAX      -34.0  -29.20      2     SZ0
1 1864-02-08  SZ000006717     TMAX      -17.4    0.68      2     SZ0
2 1864-02-17  CA006158350     TMAX      -16.7    1.94      2     CA0
3 1864-01-03  SZ000006717     TMAX      -16.5    2.30      1     SZ0
4 1864-01-02  CA006158350     TMAX      -16.1    3.02      1     CA0

In [216]:

loc.columns=['station','latitude','longitude','elevation','state','name','gsn flag', 'wmo id']
print(loc.head(3))
loc_merge=pd.merge(loc, ver, on='station', how='inner')
loc_merge

ValueError: Length mismatch: Expected axis has 8 elements, new values have 9 elements

## Exercise Section 7.2: Traffic data in Copenhagen

In this second part of exercise set 7 you will be working with traffic data from Copenhagen Municipality.

The municipality have made the data openly available through the [opendata.dk](http://www.opendata.dk/) platform. We will use the data from traffic counters to construct a dataset of hourly traffic. We will use this data to get basic insights on the development in traffic over time and relate it to weather. The gist here is to practice a very important skill in Data Science: being able to quickly fetch data from the web and structure it so that you can work with it. Scraping usually gets a bit more advanced than what we will do today, but the following exercises should give you a taste for how it works. The bulk of these exercise, however, revolve around using the Pandas library to structure and analyze data.

### Part 7.2.a: getting some data to work with

Hence follows a simple scraping exercise where you (1) collect urls for datasets in the webpage listing data on traffic counters and (2) use these urls to load the data into one dataframe.

> **Ex. 7.2.1:** Using the requests module, extract the html markup of the webpage data.kk.dk/dataset/faste-trafiktaellinger and store it as a string in a new variable.

In [92]:
import requests

tra=requests.get('https://data.kk.dk/dataset/faste-trafiktaellinger')


> **Ex. 7.2.2:** Using the re module, extract a list of all the urls in the html string and store them in a new variable.

> _Hint:_ Try using the re.findall method. You may want to Google around to figure out how to do this. Protip: searching for something along the lines of "extract all links in html regex python" and hitting the first StackOverflow link will probably get you farther than reading elaborate documentation.

In [106]:
import re
urls = re.findall(r'href=[\'"]?([^\'" >]+)', tra.text)
print (', '.join(urls))
type(urls)

/fanstatic/vendor/:version:2017-02-24T11:53:25/select2/select2.css, /fanstatic/css/:version:2017-02-24T11:53:25/main.min.css, /fanstatic/vendor/:version:2017-02-24T11:53:25/font-awesome/css/font-awesome.min.css, /fanstatic/vendor/:version:2017-02-24T11:53:25/font-awesome/css/font-awesome-ie7.min.css, /fanstatic/ckanext-geoview/:version:2017-11-10T15:39:58/css/geo-resource-styles.css, /fanstatic/ckanext-harvest/:version:2016-02-03T11:14:04/styles/harvest.css, /fanstatic/cphopendata/:version:2016-07-01T10:53:26/style.css, /favicon.ico, #content, /user/login, /user/register, /, /dataset, /organization, /group, /about, /, /organization, /organization/its, /dataset/faste-trafiktaellinger, /dataset/faste-trafiktaellinger, /dataset/groups/faste-trafiktaellinger, /dataset/activity/faste-trafiktaellinger, /dataset/faste-trafiktaellinger/resource/dcb49b51-1a17-40f6-82b4-34eaf58bce57, #, /dataset/faste-trafiktaellinger/resource/dcb49b51-1a17-40f6-82b4-34eaf58bce57, https://data.kk.dk/dataset/5f93

list

> **Ex. 7.2.3:** Create a new variable that only contains the links that point to downloadable traffic data sheets. 

> _Hint:_ You want to filter the results from above. For example to only include urls with the term 'download' in them.

In [113]:
dwl=[]
for i in urls:
    if 'download' in i:
        dwl.append(i)
print(dwl)     

['https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/dcb49b51-1a17-40f6-82b4-34eaf58bce57/download/faste-trafiktaellinger-2005.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/0bfc0425-0b2f-43b4-be71-117b4265598d/download/faste-trafiktaellinger-2006.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/e4f321ad-76bc-483f-839e-e04cfea6bfc0/download/faste-trafiktaellinger-2008.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/bac5c582-bdd0-45db-b174-f835e82851bc/download/faste-trafiktaellinger-2009.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/a209aef9-802c-4cda-9b42-f4364877b04f/download/faste-trafiktaellinger-2010.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/8b2f60a9-17ac-437d-ae6e-ad44c0a49edd/download/faste-trafiktaellinger-2011.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/reso

> **Ex. 7.2.4:** Using pd.read_excel method, load the datasets into a list. Your resulting variable should hold a list of Pandas dataframes.

> _Hint:_ you may want to set the skiprows= keyword argument. 

In [133]:
dwllist=[]
for i in dwl:
    dwllist.append(pd.read_excel(i, header=10))


> **Ex. 7.2.5:** Merge the list of dataframes into a single dataframe.

In [135]:
super=pd.concat(dwllist, join='inner')
super

Vej-Id             Vejnavn  Spor  (UTM32)  (UTM32).1  \
0      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
1      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
2      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
3      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
4      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
5      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
6      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
7      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
8      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
9      101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
10     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
11     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
12     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
13     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
14     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
15     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
16     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
17     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
18     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
19     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
20     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
21     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
22     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
23     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
24     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
25     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
26     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
27     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
28     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
29     101 1011492-0 1/ 124 -        Ellebjergvej   NaN   720881    6172777   
...                       ...                 ...   ...      ...        ...   
13681  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13682  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13683  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13684  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13685  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13686  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13687  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13688  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13689  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13690  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13691  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13692  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13693  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13694  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13695  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13696  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13697  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13698  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    6171542   
13699  101 1018541-0 3/ 638 T  Ørestads Boulevard   NaN   725458    61715

### Part 7.2.b Structuring your data

If you successfully completed the previous part, you should now have a dataframe with about 183.397 rows (if your number of rows is close but not the same, worry not—it matters little in the following). Well done! But the data is still in no shape for analysis, so we must clean it up a little.

161.236 rows (and 30 columns) is a lot of data. ~3.3 MB by my back-of-the-envelope calculations, so not "Big Data", but still enough to make your CPU heat up if you don't use it carefully. Pandas is built to handle fairly large dataframes and has advanced functionality to perform very fast operations even when the size of your data grows huge. So instead of working with basic Python we recommend working pandas built-in procedures as they are constructed to be fast on dataframes.

Nerd fact: the reason pandas is much faster than pure Python is that dataframes access a lower level programming languages (namely C, C++) which are multiple times faster than Python. The reason it is faster is that it has a higher level of explicitness and thus is more difficult to learn and navigate.

> **Ex. 7.2.6:** Reset the row indices of your dataframe so the first index is 0 and the last is whatever the number of rows your dataframe has. Also drop the column named 'index' and the one named `Spor`.

In [156]:
super_1=super.reset_index().drop(['index','Spor'], axis=1)

> **Ex. 7.2.7:** Rename variables from Danish to English using the dictionary below.

```python 
dk_to_uk = {
    'Vejnavn':'road_name',
    '(UTM32)':'UTM32_north',
    '(UTM32).1':'UTM32_east',
    'Dato':'date',
    'Vej-Id':'road_id'
}
```

In [158]:
dk_to_uk = {
    'Vejnavn':'road_name',
    '(UTM32)':'UTM32_north',
    '(UTM32).1':'UTM32_east',
    'Dato':'date',
    'Vej-Id':'road_id'
}
super_2=super_1.rename(columns=dk_to_uk)

road_id           road_name  UTM32_north  UTM32_east  \
0       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
1       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
2       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
3       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
4       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
5       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
6       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
7       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
8       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
9       101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
10      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
11      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
12      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
13      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
14      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
15      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
16      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
17      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
18      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
19      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
20      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
21      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
22      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
23      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
24      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
25      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
26      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
27      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
28      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
29      101 1011492-0 1/ 124 -        Ellebjergvej       720881     6172777   
...                        ...                 ...          ...         ...   
161206  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161207  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161208  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161209  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161210  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161211  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161212  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161213  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161214  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161215  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161216  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161217  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161218  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161219  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161220  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161221  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161222  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161223  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     6171542   
161224  101 1018541-0 3/ 638 T  Ørestads Boulevard       725458     61715

Python is quite efficient. For example, when you create a new dataframe by manipulating an old one, Python notices that—apart from some minor changes—these two objects are almost the same. Since memory is a precious resource, Python will represent the values in the new dataframe as references to the variables in the old dataset. This is great for performance, but if you for whatever reason change some of the values in your old dataframe, values in the new one will also change—and we don't want that! Luckily, we can break this dependency.

> **Ex. 7.2.8:** Break the dependencies of the dataframe that resulted from Ex. 7.2.7 using the `.copy` method. Delete all other dataframes.

In [ ]:
super_a=super_2.copy()

In [171]:
del super
del super_1
del super_2

NameError: name 'super_2' is not defined

Once you have structured appropriately, something that you will want to do again and again is selecting subsets of the data. Specifically, it means that you select specific rows in the dataset based on some column values.

>**Ex. 7.2.9:** Create a new column in the dataframe called total that is True when the last letter of road_id is T and otherwise False.

> _Hint:_ you will need the `pd.Series.str` attribute for this.

In [177]:
super_a['total']=pd.Series.str(super_a['road_id']).contains('T')


> **Ex. 7.2.10:** Select rows where total is True. Delete all the remaining observations.

In [184]:
super_b=super_a[super_a.total==True]

> **Ex. 7.2.11:** Make two datasets based on the lists of columns below. Call the dataset with spatial columns data_geo and the other data.

```python
# Columns for `geo_data`, stored in `geo_columns`
spatial_columns = ['road_name', 'UTM32_north', 'UTM32_east']

# Columns for `data`, stored in `select_columns`
hours = ['kl.{}-{}'.format(str(h).zfill(2), str(h+1).zfill(2)) for h in range(24)]
select_columns = ['road_name', 'date'] + hours
```

In [187]:
# GEO
spatial_columns = ['road_name', 'UTM32_north', 'UTM32_east']
geo_data=pd.concat([super_a,spatial_columns], join='inner', axis=0)

# DATA
hours = ['kl.{}-{}'.format(str(h).zfill(2), str(h+1).zfill(2)) for h in range(24)]
select_columns = ['road_name', 'date'] + hours

TypeError: cannot concatenate object of type "<class 'list'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

> **Ex. 7.2.12:** Drop the duplicate rows in data_geo.

In [ ]:
# [Answer to Ex. 7.2.12]

**Formatting: wide and narrow format**

When talking about two-dimensional data (matrices, tables or dataframes, we can call it many things), we can either say that it is in wide or long format (see explanation here, "wide" and "long" are used interchangably). In Pandas we can use the commands stack and unstack to move between these formats.

The wide format has the advantage that it often requires less storage and is easier to read when printed. On the other hand the long format can be easier for modelling, because each observation has its own row. Turns out that the latter is what we most often need.

> **Ex. 7.2.13:** Turn the dataset from wide to long so hourly data is now vertically stacked. Store this dataset in a dataframe called data. Name the column with hourly information hour_period. 

> _Hint:_ pandas' melt function may be of use.

In [ ]:
# [Answer to Ex. 7.2.13]

**Categorical data**

Categorical data can contain Python objects, usually strings. These are smart if you have variables with string observations that are long and often repeated, e.g. with road names.

> **Ex. 7.2.14:** Use the `.astype` method to convert the type of the road_name column to categorical.

In [ ]:
# [Answer to Ex. 7.2.14]

### Structure temporal data

Pandas has native support for working with temporal data. This is handy as much 'big data' often has time stamps which we can make Pandas aware of. Once we have encoded temporal data it can be used to extract information such as the hour, second etc.

> **Ex. 7.2.15:** Create a new column called hour which contains the hour-of-day for each row. 

In [ ]:
# [Answer to Ex. 7.2.15]

>**Ex. 7.2.16:** Create a new column called time, that contains the time of the row in datetime format. Delete the old temporal columns (hour, hour_period, date) to save memory.

> _Hint:_ try making an intermediary series of strings that has all temporal information for the row; then use pandas to_datetime function where you can specify the format of the date string.

In [ ]:
# [Answer to Ex. 7.2.16]

> **Ex. 7.2.17:** Using your time column make a new column called weekday which stores the weekday (in values between 0 and 6) of the corresponding datetime.

> _Hint:_ try using the dt method for the series called time; dt has some relevant methods itself.

In [ ]:
# [Answer to Ex. 7.2.17]

### Statistical descriptions of traffic data

> **Ex. 7.2.18:** Print the "descriptive statistics" of the traffic column. Also show a kernel density estimate of the values.

> _Hint:_ Use the describe method of pandas dataframes for the first task. Use seaborn for the second. 

In [ ]:
# [Answer to Ex. 7.2.18]

> **Ex. 7.2.19:** Which road has the most average traffic?

In [ ]:
# [Answer to Ex. 7.2.19]

> **Ex. 7.2.20:** Compute annual, average road traffic during day hours (9-17). Which station had the least traffic in 2013? Which station has seen highest growth in traffic from 2013 to 2014?

In [58]:
# [Answer to Ex. 7.2.19]